In [ ]:
#week 8
import numpy as np
import pandas as pd
import scipy as sp

class RegressionModel:
    def __init__(self, x, y, create_intercept=True, regression_type="ols", results = None):
        # Store exogenous and endogenous variables
        self.x = x
        self.y = y
        self.create_intercept = create_intercept
        self.regression_type = regression_type
        self.results = results

        # Add intercept if specified
        if self.create_intercept:
            self.add_intercept()

    def add_intercept(self):
        """Add a column of ones to the x data frame to represent the intercept."""
        self.x['intercept'] = 1

    def ols_regression(self):
        """Perform OLS regression using the x and y data frames with optimized scipy functions."""
        # Convert the x DataFrame to a NumPy matrix
        X = self.x.values

        # OLS estimation: (X'X)^(-1)X'y using scipy.linalg.solve for efficiency
        XtX = np.dot(X.T, X)  # X'X
        XtY = np.dot(X.T, self.y)  # X'y
        beta_hat = sp.linalg.solve(XtX, XtY, assume_a='pos')  # Efficient solver instead of inv(XtX)

        # Predicted values and residuals
        y_hat = np.dot(X, beta_hat)
        residuals = self.y - y_hat

        # Calculate degrees of freedom and residual variance
        n = len(self.y)  # number of observations
        p = X.shape[1]  # number of parameters (including intercept)
        df = n - p
        residual_variance = np.sum(residuals**2) / df

        # Standard errors
        var_beta_hat = residual_variance * sp.linalg.inv(XtX)  # Variance-covariance matrix of beta_hat
        standard_errors = np.sqrt(np.diag(var_beta_hat))  # Standard errors

        # t-values and p-values (using t-distribution assumption)
        t_stats = beta_hat / standard_errors
        p_values = 2 * (1 - sp.stats.t.cdf(np.abs(t_stats), df))  # Using scipy's t-distribution CDF

        # Store results in the required dictionary format
        self.results = {}
        for i, var in enumerate(self.x.columns):
            self.results[var] = {
                'intercept': 1 if var == 'intercept' else 0,
                'coefficient': beta_hat[i],
                'standard_error': standard_errors[i],
                't_stat': t_stats[i],
                'p_value': p_values[i]
            }

    def summary(self):
        """Print a summary table of the regression results."""
        if self.results is None:
            print("No results available. Please run ols_regression first.")
            return

        # Convert results dictionary into a DataFrame for summary
        summary_table = pd.DataFrame(self.results).T
        summary_table.columns = ['intercept', 'coefficient', 'standard_error', 't_stat', 'p_value']

        # Rename columns for the summary output
        summary_table.rename(columns={
            'intercept': 'Variable name',
            'coefficient': 'coefficient value',
            'standard_error': 'standard error',
            't_stat': 't-statistic',
            'p_value': 'p-value'
        }, inplace=True)

        summary_table.index.name = 'Variable name'

        # Display the summary table
        print(summary_table)